In [58]:
import csv
import numpy as np
from pandas import read_csv
from keras.utils import np_utils 

def parse_csv():
    X = read_csv('data/training.csv') 
    A = np.array(X)
    A = A[0:,30]
    res = []
    for i in A:
        res.append([int(x) for x in i.split(' ')])
    return res 
    
x_train = parse_csv()
y_train = [1] * len(x_train)
y_train = np_utils.to_categorical(y_train, 2) 

In [48]:
from PIL import Image

def read_jpeg(path):
    im = Image.open(path).convert('L')
    X = list(im.getdata())
    X = np.array(X).reshape(9216, 1)
    return X
    
x_test = read_jpeg('data/face.jpg')
y_test = []
y_test.append(1)

In [55]:
import os
import scipy.io as sio
import scipy

def read_jpeg_training_data():
    x_train = []
    y_train = []
    dirpath = 'data/'
    img_ext = '.jpg' 
    img_names = [ os.path.join(dirpath,x) for x in os.listdir( dirpath ) if x.endswith(img_ext) ]
    for i in img_names:
        x_train.append(read_jpeg(i))
        y_train.append(1)
    return x_train, y_train    

x_train, y_train = read_jpeg_training_data()
y_train = np_utils.to_categorical(y_train, 2) 

In [56]:
print(len(x_train[1]))

9216


In [59]:
height, width, deph = 96,96,1
batch_size = 128 
num_epochs = 1 
hidden_size = 512 
num_classes = 2 
conv_depth_1 = 2
pool_size = (2, 2)
kernel_size = 2
drop_prob_1 = 0.25
drop_prob_2 = 0.5

In [60]:
import numpy

x_train = numpy.array(x_train)
x_train = x_train.astype('float32')
x_train /= 225


Создание модели

In [16]:
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Activation
from keras.utils import np_utils
from keras.models import Model, Sequential  

inp = Input(shape=(1, height, width))
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
drop_1 = Dropout(drop_prob_1)(conv_1)
flat_1=Flatten()(drop_1)

hidden_1 = Dense(batch_size, activation='relu')(flat_1)
drop_2 = Dropout(drop_prob_2)(hidden_1)
out = Dense(num_classes, activation='softmax')(drop_2)

model = Model(input=inp, output=out)

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

INFO (theano.gof.compilelock): Refreshing lock /home/amir/.theano/compiledir_Linux-4.9--ARCH-x86_64-with-arch--3.5.2-64/lock_dir/lock


ImportError: /home/amir/.theano/compiledir_Linux-4.9--ARCH-x86_64-with-arch--3.5.2-64/tmpv_ykksfa/m9266a85f6dfd2aa7985e7331a643ff4e.so: undefined symbol: _ZdlPvm

In [61]:

height, width, deph = 96,96,1 #параметры изображение 28x28 в сером спектре
batch_size = 128 # рассматриваем 128 обучающий парметров на каждой итерации
num_epochs = 1 # итерируемся 20 раз по обучающей выборке 
hidden_size = 512 # по 512 нейронов в каждом (из двух) слоев НС
num_classes = 2 # 10 классов

from keras.layers import Input, Dense # 2 типа слоев НС
from keras.models import Model  #класс для определения и обучений НС

inp = Input(shape=(9216,)) #на входе вектор размера 784
hidden_1 = Dense(hidden_size, activation ='relu')(inp) #первый скрытый ReLu слой
hidden_2 = Dense(hidden_size, activation = 'relu')(hidden_1) #второй скрытый ReLu слой
out = Dense(num_classes, activation='softmax')(hidden_2) # выходной softmax слой
# softmax превращает вектор действительных чисел в вектор вероятностей

model = Model(input = inp, output = out)
#model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.compile(loss='categorical_crossentropy', #функция потерь крос - энтропии
             optimizer = 'adam', #отпимизатор Адама
              metrics = ['accuracy'] #точность - доля входных данных, отнесенных к правильному классу
             )

Обучение модели

In [63]:
model.fit(x_train, y_train, batch_size = batch_size, nb_epoch = num_epochs, verbose=1)

INFO (theano.gof.compilelock): Refreshing lock /home/amir/.theano/compiledir_Linux-4.9--ARCH-x86_64-with-arch--3.5.2-64/lock_dir/lock
ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: DimShuffle{x,x}(TensorConstant{0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/amir/anaconda3/lib/python3.5/site-packages/theano/gof/opt.py", line 1772, in process_node
    replacements = lopt.transform(node)
  File "/home/amir/anaconda3/lib/python3.5/site-packages/theano/tensor/opt.py", line 5825, in constant_folding
    no_recycling=[])
  File "/home/amir/anaconda3/lib/python3.5/site-packages/theano/gof/op.py", line 970, in make_thunk
    no_recycling)
  File "/home/amir/anaconda3/lib/python3.5/site-packages/theano/gof/op.py", line 879, in make_c_thunk
    output_storage=node_output_storage)
  File "/home/amir/anaconda3/lib/python3.5/site-packages/theano/gof/cc.py", line 1200, in make_thun

ImportError: /home/amir/.theano/compiledir_Linux-4.9--ARCH-x86_64-with-arch--3.5.2-64/tmpyw2arze1/m072ede5c7fa6c4543a7174c4ddea815a.so: undefined symbol: _ZdlPvm

In [7]:
model.evaluate(x_test, y_test, verbose=1) #оценка модели на тестовом наборе 

NameError: name 'x_test' is not defined

In [158]:
y_test = np_utils.to_categorical(y_test, 2) 
x_test = numpy.array(x_test)
x_test = x_test.reshape(1, 9216)
x_test = x_test.astype('float32')
x_test /= 225